# 装饰器

In [20]:
# 无法装饰类函数
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func

    def __call__(self, *args):
        self.calls += 1
        print(f'call {self.calls} to {self.func}')
        self.func(*args)

In [21]:
@tracer
def spam(a, b, c):
    print(a + b + c)


print(spam(1, 2, 3))
print(spam('a', 'b', 'c'))

call 1 to <function spam at 0x109b8a020>
6
None
call 2 to <function spam at 0x109b8a020>
abc
None


In [22]:
def tracer2(func):
    def on_call(*args):
        on_call.calls += 1
        print(f'call {on_call.calls} to {func.__name__} in tracer2')
        return func(*args)

    on_call.calls = 0
    return on_call

In [23]:
class C:
    @tracer2
    def spam(self, a, b, c):
        print(a + b + c)

In [24]:
x = C()
print(x.spam(1, 2, 3))
print(x.spam('a', 'b', 'c'))

call 1 to spam in tracer2
6
None
call 2 to spam in tracer2
abc
None


In [25]:
print(spam(1, 2, 3))
print(spam('a', 'b', 'c'))

call 3 to <function spam at 0x109b8a020>
6
None
call 4 to <function spam at 0x109b8a020>
abc
None


## 类装饰器

In [26]:
def count(aClass):
    aClass.count = 0
    return aClass


@count
class C: pass


In [27]:
c = C()
c.count

0

In [28]:
def decorator(cls):  # On @ decoration
    class Proxy:
        def __init__(self, *args):  # On instance creation: make a cls 
            self.wrapped = cls(*args)

        def __getattr__(self, name):  # On attribute fetch: extra ops here
            return getattr(self.wrapped, name)

    return Proxy


@decorator
class C: pass

In [29]:
x = C()

# Super

In [37]:
class A:
    def act(self):
        print('A')


class B:
    def act(self):
        print('B')


class C(A):
    def act(self):
        super().act()


class D(A, B):
    def act(self):
        super().act()


class E(B, A):
    def act(self):
        super().act()

In [38]:
c = C()
d = D()
e = E()

In [39]:
c.act()

A


In [40]:
d.act()

A


In [41]:
e.act()

B


## 运⾏时类修改与 super

In [46]:
class X:
    def m(self): print('X.m')


class Y:
    def m(self): print('Y.m')


class Z(X):
    def m(self): super().m()

In [47]:
i = Z()
i.m()

X.m


In [49]:
Z.__bases__ = (Y,)

In [50]:
i.m()

Y.m


## 协同 super 调⽤实例

In [56]:
class A:
    def __init__(self):
        print('A.__init__')


class B:
    def __init__(self):
        print('B.__init__')


class C(A, B): pass

In [57]:
c = C()

A.__init__


In [58]:
class D(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)

In [59]:
d = D()

A.__init__
B.__init__


In [83]:
class A:
    def __init__(self):
        print('A.__init__')


class B(A):
    def __init__(self):
        print('B.__init__')
        A.__init__(self)


class C(A):
    def __init__(self):
        print('C.__init__')
        A.__init__(self)



In [84]:
b = B()

B.__init__
A.__init__


In [85]:
c = C()

C.__init__
A.__init__


In [86]:
class D(B, C): pass


d = D()

B.__init__
A.__init__


In [87]:
class D(B, C):
    def __init__(self):
        B.__init__(self)
        C.__init__(self)


d = D()

B.__init__
A.__init__
C.__init__
A.__init__


使用 super

In [88]:
class A:
    def __init__(self):
        print('A.__init__')


class B(A):
    def __init__(self):
        print('B.__init__')
        super().__init__()


class C(A):
    def __init__(self):
        print('C.__init__')
        super().__init__()

In [89]:
b = B()

B.__init__
A.__init__


In [90]:
c = C()

C.__init__
A.__init__


In [91]:
class D(B, C): pass


d = D()

B.__init__
C.__init__
A.__init__


In [92]:
class D(B, C):
    def __init__(self):
        super().__init__()


d = D()

B.__init__
C.__init__
A.__init__


In [93]:
B.__mro__

(__main__.B, __main__.A, object)

In [82]:
D.__mro_B

(__main__.D, __main__.B, __main__.C, __main__.A, object)

使⽤范围：⼀个“要么不⽤，要么全⽤”的模型

In [99]:
class A:
    def __init__(self):
        print('A.__init__')
        super().__init__()


class B:
    def __init__(self):
        print('B.__init__')
        super().__init__()


class C(A, B):
    def __init__(self):
        print('C.__init__')
        super().__init__()


c = C()

C.__init__
A.__init__
B.__init__


In [100]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

In [101]:
class A:
    def __init__(self):
        print('A.__init__')
        # super().__init__()


class B:
    def __init__(self):
        print('B.__init__')
        super().__init__()


class C(A, B):
    def __init__(self):
        print('C.__init__')
        super().__init__()


c = C()

C.__init__
A.__init__


In [102]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

定制方法

In [121]:
class A:
    def method(self):
        print('A.method')
        super().method()


class B(A):
    def method(self):
        print('B.method')
        super().method()


class C:
    def method(self):
        print('C.method')


class D(B, C):
    def method(self):
        print('D.method')
        super().method()

In [122]:
d = D()
d.method()

D.method
B.method
A.method
C.method


In [123]:
D.__mro__

(__main__.D, __main__.B, __main__.A, __main__.C, object)

In [124]:
class B(A):
    def method(self):
        print('B.method')


class D(B, C):
    def method(self):
        print('D.method')
        super().method()

In [125]:
d = D()
d.method()

D.method
B.method


耦合：mixin 类的应⽤

只要⽅法名集合是不相交的，这刚好就能⼯作。每个分支的调⽤链能够独⽴地存在

In [134]:
class A:
    def other(self):
        print('A.other')


class Mixin(A):
    def other(self):
        print('Mixin.other')
        super().other()


class B:
    def method(self):
        print('B.method')


class C(Mixin, B):
    def method(self):
        print('C.method')
        super().other()
        super().method()

In [135]:
c = C()
c.method()

C.method
Mixin.other
A.other
B.method


In [136]:
class C(B, Mixin):
    def method(self):
        print('C.method')
        super().other()
        super().method()


c = C()
c.method()

C.method
Mixin.other
A.other
B.method


In [137]:
class A:
    def other(self):
        print('A.other')


class Mixin(A):
    def other(self):
        print('Mixin.other')
        A.other(self)


class B:
    def method(self):
        print('B.method')


class C(Mixin, B):
    def method(self):
        print('C.method')
        Mixin.other(self)
        B.method(self)

In [138]:
c = C()
c.method()

C.method
Mixin.other
A.other
B.method


对于钻石模式中类似的同名方法分发顺序也许就没有那么凑巧了。例如在一个像前面的钻石模式中，一个客户类有可能让一个 super 调用的企图无效化

In [152]:
class A:
    def method(self):
        print('A.method')


class Mixin(A):
    def method(self):
        print('Mixin.method')
        super().method()


class B(A):
    def method(self):
        print('B.method')


class C(Mixin, B):
    def method(self):
        print('C.method')
        super().method()

In [159]:
Mixin().method()

Mixin.method
A.method


In [154]:
C().method()  # ⽆法使⽤ A.method

C.method
Mixin.method
B.method


In [158]:
C.__mro__

(__main__.C, __main__.Mixin, __main__.B, __main__.A, object)


类 A 定义了一个 method。
类 Mixin 也定义了一个 method 并且在其内部调用了 super().method()，这会进一步调用 A.method。
类 B 同样定义了自己的 method。
类 C 继承自 Mixin 和 B，并且重写了 method，在其中调用了 super().method()。
关键在于理解 super().method() 的调用链：
当你执行 C().method() 时，首先打印出 "C.method"。
接着，C 的 method 内部调用 super().method()。由于 C 的 MRO 是 [C, Mixin, B, A, object]，super() 会找到 Mixin 类中的 method。
Mixin 的 method 打印出 "Mixin.method"，然后它内部的 super().method() 调用会继续沿着 MRO 到达 B 类的 method，因为 B 在 A 之前。
B 的 method 打印出 "B.method"，但请注意，B 的 method 并没有再次调用 super().method()，所以调用链在此停止，并不会自动继续到 A.method。

In [151]:
class A:
    def method(self):
        print('A.method')


class Mixin(A):
    def method(self):
        print('Mixin.method')
        A.method(self)


class B(A):
    def method(self):
        print('B.method')


class C(Mixin, B):
    def method(self):
        print('C.method')
        super().method()


C().method()

C.method
Mixin.method
A.method


因为 MRO可以随着类树变化，super 在不同的类树中会调⽤同⼀⽅法的不同版本—有些甚⾄可能在你 编写⼀个类的时候都不会预料到：

In [160]:
class Employee:
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary


class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 30000)


class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 20000)

In [161]:
sue, bob = Server('Sue Jones'), Chef('Bob Smith')
sue.salary, bob.salary

(20000, 30000)

In [175]:
class Twojobs(Server, Chef): pass

In [178]:
tom = Twojobs('Tom')
tom.salary

20000

In [177]:
Twojobs.__mro__

(__main__.Twojobs, __main__.Server, __main__.Chef, __main__.Employee, object)

super()调用

In [180]:
class Chef2(Employee):
    def __init__(self, name):
        super().__init__(name, 30000)


class Server2(Employee):
    def __init__(self, name):
        super().__init__(name, 20000)

In [181]:
sue, bob = Server2('Sue Jones'), Chef2('Bob Smith')
sue.salary, bob.salary

(20000, 30000)

In [182]:
class Twojobs2(Server2, Chef2): pass

In [184]:
tom = Twojobs2('Tom')
tom.salary

TypeError: Chef2.__init__() takes 2 positional arguments but 3 were given

In [185]:
Twojobs2.__mro__

(__main__.Twojobs2,
 __main__.Server2,
 __main__.Chef2,
 __main__.Employee,
 object)

首先调用父类的__init__()方法，找到 Server2类的__init()__方法，在其中的 super()调用找到 Chef2类的__init__()方法，传入参数为 Chef2(self, name, 20000) 与参数列表不符

## 类陷阱
因为所有从类产⽣的实例都共享这个类的命名空间，所以 任何在类层次所做的修改都会反映在所有实例中，除⾮实例拥有⾃⼰的修改过的类属性版本。

In [187]:
class X: a = 1

In [188]:
A = X()
A.a

1

In [189]:
X.a = 3

In [190]:
A.a

3

In [191]:
A.a = 5

In [192]:
X.a

3

In [193]:
A.a

5

修改可变类属性也可能产⽣副作⽤
由于类属性被所有实例共享，因此如果引⽤⼀个可变对象，那么从任何实例在原位置修改该对象都会⽴刻影响到所有实例

In [201]:
class C:
    shared = []

    def __init__(self):
        self.private = []

In [202]:
x, y = C(), C()

In [203]:
x.shared, y.shared, C.shared

([], [], [])

In [204]:
x.shared.append(1)

In [205]:
x.shared, y.shared, C.shared

([1], [1], [1])

In [206]:
x.private.append(2)
x.private, y.private

([2], [])

In [208]:
x.shared, y.shared, C.shared


([1], [1], [1])

根据方法解析顺序（Method Resolution Order, MRO）来决定调用哪个父类的方法

In [244]:
class A:
    def __str__(self): ...

    def other(self): ...


class B:
    def __str__(self): ...

    def other(self): ...


class C(A, B): ...


# 此时C继承的属性都为A
class D(B, A): ...


# 此时 D 继承的属性都为 B
class E(A, B): __str__ = B.__str__

# 指定继承父类 B 中的 str 方法

In [246]:
# 获取 other 方法定义所在的类名
def get_method_class_name(instance, method_name):
    for cls in instance.__class__.mro():
        if method_name in cls.__dict__:
            return cls.__name__
    return None


c = C()
print(get_method_class_name(c, '__str__'))
d = D()
print(get_method_class_name(d, '__str__'))
e = E()
print(get_method_class_name(e, '__str__'))

A
B
E


In [250]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

In [251]:
C.mro()

[__main__.C, __main__.A, __main__.B, object]

# 练习

In [258]:
lst1 = [1, 2, 3]
lst2 = [4, 5, 6]
lst1 + lst2

[1, 2, 3, 4, 5, 6]

In [263]:
dic1 = {'a': 1, 'b': 2}
dic2 = {'a': 3, 'd': 4}
dic1.update(dic2)
dic1

{'a': 3, 'b': 2, 'd': 4}

In [271]:
class Adder:
    def __init__(self, data):
        self.data = data
        
    def __add__(self, other):
        self.add(self.data, other)
        
    def add(self, obj1, obj2):
        print('Not implemented')

class ListAdder(Adder):
    def add(self, lst1, lst2):
        return lst1 + lst2

class DictAdder(Adder):
    def add(self, dct1, dct2):
        return dct1.update(dct2)
    
d = DictAdder([1,2,[3]])
d.copy()

AttributeError: 'DictAdder' object has no attribute 'copy'

In [308]:
class MyList:
    def __init__(self, iterable=None):
        """构造函数，接受一个可迭代对象（如列表、元组等）或另一个 MyList 实例，将其内容复制到新实例中"""
        if iterable is None:
            self._data = []
        elif isinstance(iterable, list):
            # self._data = iterable.copy()
            self._data = iterable[:]
        elif isinstance(iterable, MyList):
            self._data = iterable._data[:]
        else:
            try:
                self._data = list(iterable)
            except TypeError as e:
                raise ValueError("Iterable must be a list-like object") from e 

    def __getattr__(self, name):
        # 如果 MyList 类没有定义 name 这个属性或方法，则尝试从内部列表 _data 获取
        print('__getattr__')
        if hasattr(self._data, name):
            return getattr(self._data, name)
        else:
            raise AttributeError(f"'MyList' object has no attribute '{name}'")
    def __setattr__(self, name, value):
        # 设置属性时，区分是否设置 _data 这个内部属性，避免无限递归
        print('__setattr__')
        if name == '_data':
            super().__setattr__(name, value)
            # object.__setattr__(self, name, value)
        else:
            # 尝试设置内部列表 _data 的属性或方法
            setattr(self._data, name, value)

    def __add__(self, other):
        """重载 '+' 运算符，用于合并两个列表"""
        if not isinstance(other, (list, MyList)):
            raise TypeError("Can only concatenate list (not \"{}\") to list".format(type(other).__name__))
        return MyList(self._data + list(other))

    def __getitem__(self, index):
        """重载索引操作，支持正索引、负索引和切片"""
        return self._data[index]

    def __setitem__(self, index, value):
        """重载索引赋值操作"""
        self._data[index] = value

    def __delitem__(self, index):
        """重载删除操作"""
        del self._data[index]

    def __iter__(self):
        """使类可迭代"""
        return iter(self._data)

    def __len__(self):
        """重载 len() 函数"""
        return len(self._data)

    def append(self, item):
        """重载列表的 append 方法"""
        self._data.append(item)

    def sort(self, *, key=None, reverse=False):
        """重载列表的 sort 方法"""
        self._data.sort(key=key, reverse=reverse)

    def __str__(self):
        return str(self._data)
    
    # 为了完整性和更好的交互性，还可以考虑重载其他魔法方法，如 __repr__, __str__, __contains__, __eq__, __iadd__, 等。

In [309]:
# 测试 MyList 类
# 创建 MyList 实例
my_list1 = MyList([1, 2, 3])
my_list2 = MyList([4, 5, 6])

# 测试运算符重载
combined = my_list1 + my_list2
print(combined)  # 应该输出 MyList 实例，内容为 [1, 2, 3, 4, 5, 6]

__setattr__
__setattr__
__setattr__
[1, 2, 3, 4, 5, 6]


In [310]:
my_list3 = MyList(my_list1)
print(my_list3)
print(my_list3._data)
my_list3.age

__setattr__
[1, 2, 3]
[1, 2, 3]
__getattr__


AttributeError: 'MyList' object has no attribute 'age'

In [295]:
# 测试索引和分片
print(my_list1[0])  # 输出 1
print(my_list1[-1])  # 输出 3
print(my_list1[1:3])  # 输出 MyList 实例，内容为 [2, 3]

# 测试迭代
for item in my_list1:
    print(item)

# 测试方法
my_list1.append(7)
print(my_list1)  # 应该输出 MyList 实例，内容为 [1, 2, 3, 7]
my_list1.sort(reverse=True)
print(my_list1)  # 应该输出 MyList 实例，内容为 [7, 3, 2, 1]


1
3
[2, 3]
1
2
3
[1, 2, 3, 7]
[7, 3, 2, 1]
